<a href="https://colab.research.google.com/github/tb-harris/neuroscience-2025/blob/main/9_Multiple_Regression_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # pandas (DataFrames)
import seaborn as sns # graphs
import statsmodels.formula.api as smf # regression models
# installing and importing the custom diagnostics module
%pip install git+https://github.com/tb-harris/diagnostics.git
from diagnostics import LinearRegDiagnostic

from google.colab import drive
drive.mount('/content/drive')

  Cloning https://github.com/tb-harris/diagnostics.git to /tmp/pip-req-build-3_p7rvkr
  Running command git clone --filter=blob:none --quiet https://github.com/tb-harris/diagnostics.git /tmp/pip-req-build-3_p7rvkr
  Resolved https://github.com/tb-harris/diagnostics.git to commit 63e7ed899813d387d104118e3b35d435b398c0df
  Preparing metadata (setup.py) ... done
  Created wheel for diagnostics: filename=diagnostics-0.1-py3-none-any.whl size=4726 sha256=299d3de60a95f4fdb1e7cc47ff057eca4d353c7f133ff6ba14fa442312797827
  Stored in directory: /tmp/pip-ephem-wheel-cache-t_66fy4r/wheels/dc/6f/7f/5d7dbbdace3319b3bc68e42a4691a9f8e3605e00b9f8b8ce2b
Successfully built diagnostics
Mounted at /content/drive


In [ ]:
cell_metadata = pd.read_csv("/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/20200711_patchseq_metadata_mouse.csv", index_col="cell_specimen_id")

rna_data = pd.read_hdf("/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/gene_counts_ppm.hdf5.lz4")

morph_additional = pd.read_csv(
    "/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/morph_features_expanded.csv",
    index_col="cell_specimen_id"
)

morph_positional = pd.read_csv(
    "/content/drive/Shareddrives/Lisman Laboratory/Lisman 2025/Neuroscience 2025/Data/morph_features_positional.csv",
    index_col="cell_specimen_id"
)

In [ ]:
morph_full_recon = morph_additional[cell_metadata["neuron_reconstruction_type"] == "full"]
gene_morph = morph_full_recon.join(rna_data)
gene_morph = gene_morph.dropna()

nonzero_genes = gene_morph.sum() != 0
gene_morph = gene_morph.loc[:, nonzero_genes]

/tmp/ipython-input-6-2565288093.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  morph_full_recon = morph_additional[cell_metadata["neuron_reconstruction_type"] == "full"]


In [ ]:
gene_only = gene_morph.iloc[:, 159:]